In [1]:
import pyspark

In [22]:
conf = pyspark.SparkConf()
conf.setMaster('spark://spark-master:7077')
spark_context = pyspark.SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=spark://spark-master:7077) created by __init__ at <ipython-input-2-9c7ba4625174>:3 

In [149]:
# Открытие текстового файла и превращение его в RDD.
# К докеру со spark-worker примаунчена папка /user-data с вашего диска, в которой лежат скачанные данные.
data = spark_context.textFile('/user-data/combined/reviews.csv')
# Берём первую строку, для дальнейшей фильтрации. Первая строка отвечает за заголовки, они нас не интересуют.
header_data = data.first()
# Начались вычисления. Первая функция пропускает все данные кроме первой строки с заголовками.
# Функция `map` разделяет строку на колонки, используя разделитель `,`.
result_data = data.filter(lambda row: row != header_data) \
    .map(lambda line: line.split(","))

In [150]:
result_data.take(10)

[['bj',
  '0_bj',
  'Ilovebennjerry',
  '2017-04-15',
  '3',
  'Not enough brownies!',
  '10.0',
  '3.0',
  '"Super good',
  " don't get me wrong. But I came for the caramel and brownies",
  " not the sweet cream. The packaging made it seem like brownies were packed and bountiful *crying frowny emoji* I'd say the taste of this was amazing",
  " but the ratio of brownie to sweet cream was disappointing. Liked it regardless but probably won't buy again simply because it didn't live up to its promising package. I'll find another one that has a better ratio and wayyy more yummy chewy brownies."],
 ['Overall',
  ' good flavor',
  ' texture',
  ' idea',
  ' and brownies. Not so great caramel/sweet cream/ brownie RATIO. Just add more brownies. Please."',
  '',
  '',
  '',
  ''],
 ['bj',
  '0_bj',
  'Sweettooth909',
  '2020-01-05',
  '5',
  'I’m OBSESSED with this pint!',
  '3.0',
  '0.0',
  '"I decided to try it out although I’m not a huge caramel fan',
  ' and the first buy was ok',
  ' didn

In [151]:
result_data = result_data.filter(lambda line: len(line) > 5)

In [152]:
result_data = result_data.filter(lambda line: line[4].isdigit())

In [153]:
result_data.take(10)

[['bj',
  '0_bj',
  'Ilovebennjerry',
  '2017-04-15',
  '3',
  'Not enough brownies!',
  '10.0',
  '3.0',
  '"Super good',
  " don't get me wrong. But I came for the caramel and brownies",
  " not the sweet cream. The packaging made it seem like brownies were packed and bountiful *crying frowny emoji* I'd say the taste of this was amazing",
  " but the ratio of brownie to sweet cream was disappointing. Liked it regardless but probably won't buy again simply because it didn't live up to its promising package. I'll find another one that has a better ratio and wayyy more yummy chewy brownies."],
 ['bj',
  '0_bj',
  'Sweettooth909',
  '2020-01-05',
  '5',
  'I’m OBSESSED with this pint!',
  '3.0',
  '0.0',
  '"I decided to try it out although I’m not a huge caramel fan',
  ' and the first buy was ok',
  ' didn’t like the caramel too much and for some reason that specific pint barely had any brownies! Like there were some on top but from the middle to the bottom? Zilch. Nada. Nothing! It wa

In [154]:
result_data = result_data.map(lambda line: (line[1], int(line[4]), line[6], line[7]))

In [155]:
result_data.take(10)

[('0_bj', 3, '10.0', '3.0'),
 ('0_bj', 5, '3.0', '0.0'),
 ('0_bj', 3, '5.0', '2.0'),
 ('0_bj', 5, '24.0', '1.0'),
 ('0_bj', 1, '1.0', '5.0'),
 ('0_bj', 2, '3.0', '1.0'),
 ('0_bj', 3, ' but..."', '3.0'),
 ('0_bj', 3, '4.0', '1.0'),
 ('0_bj', 2, '8.0', '6.0'),
 ('0_bj', 3, '1.0', '1.0')]

In [156]:
  def calculate_weight(helpful_yes, helpful_no):
        try:
            hy = float(helpful_yes)
        except:
            return 0
        try:
            hn = float(helpful_no)
        except:
            return 0
        if (hy + hn) ==0:
            return 0
        return hy / (hy + hn)


In [157]:
result_data = result_data.map(lambda line: (line[0], line[1], calculate_weight(line[2], line[3])))

In [158]:
result_data.take(10)

[('0_bj', 3, 0.7692307692307693),
 ('0_bj', 5, 1.0),
 ('0_bj', 3, 0.7142857142857143),
 ('0_bj', 5, 0.96),
 ('0_bj', 1, 0.16666666666666666),
 ('0_bj', 2, 0.75),
 ('0_bj', 3, 0),
 ('0_bj', 3, 0.8),
 ('0_bj', 2, 0.5714285714285714),
 ('0_bj', 3, 0.5)]

In [159]:
result_data = result_data.map(lambda line: (line[0], (line[1]*line[2], 1)))

In [160]:
result_data.take(10)

[('0_bj', (2.307692307692308, 1)),
 ('0_bj', (5.0, 1)),
 ('0_bj', (2.142857142857143, 1)),
 ('0_bj', (4.8, 1)),
 ('0_bj', (0.16666666666666666, 1)),
 ('0_bj', (1.5, 1)),
 ('0_bj', (0, 1)),
 ('0_bj', (2.4000000000000004, 1)),
 ('0_bj', (1.1428571428571428, 1)),
 ('0_bj', (1.5, 1))]

In [161]:
result_data = result_data.reduceByKey(lambda val1, val2: (val1[0] + val2[0], val1[1] + val2[1]))

In [162]:
result_data.take(10)

[('0_bj', (94.36007326007326, 208)),
 ('6_bj', (39.78703703703704, 10)),
 ('7_bj', (30.0, 7)),
 ('9_bj', (125.17807017543859, 42)),
 ('11_bj', (34.08571428571429, 9)),
 ('13_bj', (101.01336675020886, 133)),
 ('14_bj', (89.43318903318902, 153)),
 ('16_bj', (84.21103603603603, 978)),
 ('18_bj', (84.69215686274511, 111)),
 ('21_bj', (91.12058175742388, 108))]

In [163]:
result_data = result_data.mapValues(lambda x: x[0] / x[1])

In [164]:
result_data.take(10) 

[('56_hd', 1.3518518518518519),
 ('59_hd', 0.8885109666996628),
 ('60_hd', 1.5065746219592373),
 ('61_hd', 0.5623482340187146),
 ('62_hd', 0.6853603603603604),
 ('63_hd', 1.5138888888888888),
 ('66_hd', 0.5871290603239602),
 ('69_hd', 1.9545454545454546),
 ('0_talenti', 0.7385537457479904),
 ('2_talenti', 2.097386587771203)]

In [165]:
result_data.coalesce(1).saveAsTextFile('/user-data/combined/rating_by_reviews')

Py4JJavaError: An error occurred while calling o1094.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/user-data/combined/rating_by_reviews already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:289)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1090)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:388)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1088)
	at org.apache.spark.rdd.PairRDDFunc

In [327]:
res_schema = ["key_rating", "rating_by_reviews"]

In [328]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [329]:
res_df = spark.createDataFrame(result_data, res_schema)

In [330]:
res_df.columns

['key_rating', 'rating_by_reviews']

In [366]:
prod_df = spark.read.option("header", "true").csv("/user-data/combined/products_copy.csv", quote='"', sep=",")

In [367]:
prod_schema = ['brand', 'key', 'name', 'subhead','description','rating','rating_count','ingredients']

In [368]:
prod_df.columns

['brand',
 'key',
 'name',
 'subhead',
 'description',
 'rating',
 'rating_count',
 'ingredients']

In [369]:
prod_df.show(1)

+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
|brand| key|               name|             subhead|         description|rating|rating_count|         ingredients|
+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
|   bj|0_bj|Salted Caramel Core|Sweet Cream Ice C...|Find your way to ...|   3.7|         208|CREAM, SKIM MILK,...|
+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
only showing top 1 row



In [376]:
from pyspark.sql.functions import col
#   you can't modify the existing dataframe as it is immutable, but you can return a new dataframe with the desired modifications.
new_df = prod_df.join(res_df, col('key') == col('key_rating'), "left"). drop(col('key_rating'))

In [380]:
new_df.show(5)

+------------------+--------------------+--------------------+--------------------+--------------------+------+------------+--------------------+------------------+
|             brand|                 key|                name|             subhead|         description|rating|rating_count|         ingredients| rating_by_reviews|
+------------------+--------------------+--------------------+--------------------+--------------------+------+------------+--------------------+------------------+
|Part of our secret| is the quality o...|                null|                null|                null|  null|        null|                null|              null|
|Part of our secret| is the quality o...|                null|                null|                null|  null|        null|                null|              null|
|                hd|               66_hd|Vanilla Milk Choc...|                null|We start with pur...|   2.5|         224|CREAM, SKIM MILK,...|0.5871290603239602|
|         

In [372]:
prod_df.columns

['brand',
 'key',
 'name',
 'subhead',
 'description',
 'rating',
 'rating_count',
 'ingredients']

In [373]:
prod_df.show(1)

+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
|brand| key|               name|             subhead|         description|rating|rating_count|         ingredients|
+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
|   bj|0_bj|Salted Caramel Core|Sweet Cream Ice C...|Find your way to ...|   3.7|         208|CREAM, SKIM MILK,...|
+-----+----+-------------------+--------------------+--------------------+------+------------+--------------------+
only showing top 1 row



In [378]:
new_df.write.option("header", "true").csv("/user-data/combined/products_with_ratings_7.csv")